In [5]:
# Import necessary libraries
import os
import sys
import pandas as pd
# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))

In [6]:
# Import the load_data function from the data_loader module
from data_loader import load_data

In [7]:

# Define file path
file_path = 'D:/Kifya_training/Week 3/Week 3/AlphaCare-Insurance-Analysis/notebooks/data/MachineLearningRating_v3.txt'

# Load data using the custom function
data = pd.read_csv(file_path, delimiter='|')

# Display the first few rows of the data
print(data.head())

C:\Users\Yoni\AppData\Local\Temp\ipykernel_15144\317629331.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter='|')


   UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered Citizenship          LegalType Title Language                 Bank      AccountType  MaritalStatus         Gender       Country Province  PostalCode MainCrestaZone SubCrestaZone          ItemType      mmcode        VehicleType  RegistrationYear           make  Model  Cylinders  cubiccapacity  kilowatts bodytype  NumberOfDoors VehicleIntroDate  CustomValueEstimate AlarmImmobiliser TrackingDevice CapitalOutstanding          NewVehicle WrittenOff Rebuilt Converted CrossBorder  NumberOfVehiclesInFleet  SumInsured TermFrequency  CalculatedPremiumPerTerm                    ExcessSelected CoverCategory   CoverType            CoverGroup              Section                          Product StatutoryClass StatutoryRiskType  TotalPremium  TotalClaims
0               145249     12827  2015-03-01 00:00:00             True              Close Corporation    Mr  English  First National Bank  Current account  Not specified  Not spe